# Inexact convex hull computations in Oscar.jl

This notebook illustrates what goes wrong when using inexact coordinates, i.e. `Float64` in `Oscar.jl` when computing convex hulls. Here by computing the convex hull we mean to switch from a representation of a polytope as a convex hull of finitely many points to a representation as the intersection of finitely many halfspaces.

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.9.0-DEV ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


In [2]:
C = Polyhedron(Polymake.polytope.cyclic(4,20))
P = C*C

A polyhedron in ambient dimension 8

We will repeatedly compute the convex hull of `P`, from it's vertices or facets, and with exact coordinates the polytope stays the same. The numbered output lines contain the numbers of vertices and facets. We can see that these stay the same of course, since the polytopes stay the same.

We switch to a slightly smaller `P`, since the original is too large for the exact computation.

In [6]:
Ptmp = cyclic_polytope(3,7) * cyclic_polytope(3,7)
PF = Polyhedron(facets(Ptmp))
for i in 1:5
    println("$(i): $(nvertices(PF)) $(nfacets(PF))")
    next = convex_hull(vertices(PF))
    println("$(i) next: $(nvertices(next)) $(nfacets(next))")
    println(next == PF)
    global PF = Polyhedron(facets(next))
    
end

1: 49 20
1 next: 49 20
true
2: 49 20
2 next: 49 20
true
3: 49 20
3 next: 49 20
true
4: 49 20
4 next: 49 20
true
5: 49 20
5 next: 49 20
true


Now we change our coordinate type to `Float64` and repeat the same loop. Here we see that the computation breaks down immediately, since the number of vertices and facets is wrong.

The loop cannot even finish since the programs involved sense that something is off. This is actually a good thing, but not all programs (can) check for errors due to inexactness.

In [3]:
PF = Polyhedron{Float64}(facets(P))
for i in 1:5
    Polymake.prefer("cdd"; application="polytope") do
        println("$(i): $(nvertices(PF)) $(nfacets(PF))")
        next = convex_hull(Float64, vertices(PF))
        println("$(i) next: $(nvertices(next)) $(nfacets(next))")
        global PF = Polyhedron{Float64}(facets(next))
    end
end

1: 697 228


polymake:  WARNING: rule cdd.convex_hull.canon: POINTED, RAYS, LINEALITY_SPACE : INPUT_RAYS failed: Error in dd_MatrixCanonicalize: 15
polymake:  WARNING: rule cdd.convex_hull.redund: RAYS, RAY_SEPARATORS, LINEALITY_SPACE : INPUT_RAYS failed: Error in dd_Redundant: 15
polymake:  WARNING: rule cdd.convex_hull.primal: FACETS, LINEAR_SPAN : RAYS | INPUT_RAYS failed: Error in dd_DDMatrix2Poly: 16


LoadError: Exception occured at Polymake side:
no more rules available to compute 'VERTICES'


Another example. Here the loop degenerates to something stable, nevertheless, the end result is wrong.

In [4]:
P = cyclic_polytope(5,20)
PF = Polyhedron{Float64}(facets(P))
for i in 1:5
    Polymake.prefer("cdd"; application="polytope") do
        println("$(i): $(nvertices(PF)) $(nfacets(PF))")
        next = convex_hull(Float64, vertices(PF))
        println("$(i) next: $(nvertices(next)) $(nfacets(next))")
        global PF = Polyhedron{Float64}(facets(next))
    end
end

1: 21 18
1 next: 10 42
2: 10 4
2 next: 6 6
3: 6 6
3 next: 6 6
4: 6 6
4 next: 6 6
5: 6 6
5 next: 6 6
